In [151]:
import PyPDF2
import os
import glob
import re
import ast
import itertools



directory = r'C:\Users\Sergio\Desktop\TWS\tws-workspace\closeAI\Examples\content_generation\content\nutrition'
all_content = []

# Get all PDF files in the directory
pdf_files = glob.glob(os.path.join(directory, '*.pdf'))



In [152]:
import openai

openai.api_key = ""

def get_completion_and_token_count(messages, 
                                 model="gpt-3.5-turbo-16k", 
                                 temperature=0.0, max_tokens=5000):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )

    content = response.choices[0].message["content"]


    token_dict = {
        'prompt_tokens': response['usage']['prompt_tokens'],
        'completion_tokens': response['usage']['completion_tokens'],
        'total_tokens': response['usage']['total_tokens'],
    }

    return content, token_dict

In [153]:
def find_substring(results, substrings):
    # For each substring
    for sub in substrings:
        # Store indices where the substring is found
        found_indices = []

        # For each result
        for index, sub_results in results:
            # For each substring result
            for sub_res, found in sub_results:
                # If the current substring matches and it was found
                if sub_res == sub and found:
                    # Store the index
                    found_indices.append(index)
        
        # If the substring was found at least once
        if found_indices:
            # Return the substring and the indices where it was found
            return sub, found_indices

    # If none of the substrings were found
    return None, []

In [154]:
def get_main_topics(pdf_file):
    text_content = []
    print(os.path.basename(pdf_file))

    # Open the pdf file
    with open(pdf_file, 'rb') as file:
        # Initialize a pdf file reader
        reader = PyPDF2.PdfReader(file)

        # Loop through each page
        for i in range(len(reader.pages)):
            # Get the text content of the page
            page = reader.pages[i]
            text = page.extract_text()

            # Add the text to the list
            text_content.append(text)

    substrings = ['contents', 'indice', 'index', 'índice', 'table of contents']

    text_content_first = text_content[:8]

    results = list(filter(lambda x: any(sub_res[1] for sub_res in x[1]), enumerate(map(lambda s: [(sub, s.lower().find(sub) != -1) for sub in substrings], text_content_first))))

    # Specify the substrings in the order you want to search for them
    substrings = ["table of contents", "indice", "index", "índice", "contents"]

    # Call the function with your results and substrings
    found_substring, found_indices = find_substring(results, substrings)

    print(found_substring)
    print(found_indices)

    index_location = found_indices[:1]
    index_location = int(index_location[0])

    print(text_content[index_location])


    pattern = re.compile(r"^\d+\.\s+.+", re.MULTILINE)

    all_matching_substrings = []

    matching_dict = {}

    for i in range(index_location, index_location + 3):
        matching_substrings = pattern.findall(text_content[i])

        for substring in matching_substrings:
            # Use a regular expression to separate the topic name and page number
            match = re.match(r'^(\d+)\.\s+(.+?)(?:\s+(\d+))?\s*$', substring)
            if match:
                # Store the topic name and page number in the dictionary
                matching_dict[match.group(2)] = match.group(3) if match.group(3) else None

    
    if not all_matching_substrings:
        print("OTHEEER")
        # Define a new pattern to match the other format
        pattern = re.compile(r"^(\d+(?:\.\d+)*)\s+(.+)\s+(\d+)$", re.MULTILINE)

        for i in range(index_location, index_location + 3):
            matching_substrings = pattern.findall(text_content[i])

            # Initialize a dictionary to store the topic name and page number

            for substring in matching_substrings:
                # 'substring' here is already a tuple because re.findall was used
                # So we can directly access its elements
                index, topic_name, page_number = substring
                
                # Store the topic name and page number in the dictionary
                matching_dict[f"{index} {topic_name}"] = page_number

            
    if matching_dict:
            all_matching_substrings.append(matching_dict)

    print(len(str(all_matching_substrings)))

    # Check the length of all_matching_substrings
    if len(str(all_matching_substrings)) > 1500:
        # Split the dictionary into two
        matching_dict_iter = iter(matching_dict.items())
        matching_dict_1 = dict(itertools.islice(matching_dict_iter, len(matching_dict)//2))
        matching_dict_2 = dict(matching_dict_iter)

        # Replace the original matching_dict in all_matching_substrings with the two new dictionaries
        all_matching_substrings = [matching_dict_1, matching_dict_2]

    system_message_clean = f""" 
    You are going to receive an array of dictionaries. The keys are not cleaned and not clear. \n
    The array will be between triple backstrick (```). \n
    Output the same array of dictionaries with keys strings cleaned and its number in case it has, and its respective dictionary value. \n
    Just output the object asked for. ```{all_matching_substrings}``` """

    if(len(all_matching_substrings) > 1):
        for i in all_matching_substrings:
            print(i)
            system_message_clean = f""" 
                    You are going to receive an array of dictionaries. The keys are not cleaned and not clear. \n
                    The array will be between triple backstrick (```). \n
                    Output the same array of dictionaries with keys strings cleaned and its number in case it has, and its respective dictionary value. \n
                    Just output the object asked for. ```{i}``` """
            messages =  [  
            {'role':'system', 
            'content': system_message_clean},  
            ] 
            response, token_dict = get_completion_and_token_count(messages)
            print(response)
            print(token_dict)

            # Convert the string representation of the response into an actual list of dictionaries
            response = ast.literal_eval(response)

            all_content.append({"file_name": os.path.basename(pdf_file), "matching_substrings": response})
    else:
        messages =  [  
        {'role':'system', 
        'content': system_message_clean},  
        ] 
        response, token_dict = get_completion_and_token_count(messages)
        print(response)
        print(token_dict)

        # Convert the string representation of the response into an actual list of dictionaries
        response = ast.literal_eval(response)

        all_content.append({"file_name": os.path.basename(pdf_file), "matching_substrings": response})


In [155]:
# Process all PDF files

for pdf_file in pdf_files:
    get_main_topics(pdf_file)

Introduction to Human Nutrition 2nd Edition ( PDFDrive ).pdf
index
[6, 7]
This edition ﬁ  rst published 2009
First edition published 2002
© 2009, 2002 by The Nutrition Society
Blackwell Publishing was acquired by John Wiley & Sons in February 2007. Blackwell’s publishing programme has been 
merged with Wiley’s global Scientiﬁ  c, T echnical, and Medical business to form Wiley-Blackwell.
Registered ofﬁ  ce
John Wiley & Sons Ltd, The Atrium, Southern Gate, Chichester, West Sussex, PO19 8SQ, United KingdomEditorial ofﬁ  ces
9600 Garsington Road, Oxford, OX4 2DQ, United Kingdom
2121 State Avenue, Ames, Iowa 50014-8300, USA
For details of our global editorial ofﬁ  ces, for customer services and for information about how to apply for permission to 
reuse the copyright material in this book please see our website at www.wiley.com/wiley-blackwell.The right of the author to be identiﬁ  ed as the author of this work has been asserted in accordance with the Copyright, 
Designs and Patents Act 198

In [156]:
print(all_content)

[{'file_name': 'Introduction to Human Nutrition 2nd Edition ( PDFDrive ).pdf', 'matching_substrings': [{'Introduction to Human Nutrition: A Global Perspective on Food and Nutrition': 1, 'Body Composition': 12, 'Energy Metabolism': 31, 'Nutrition and Metabolism of Proteins and Amino Acids': 49, 'Digestion and Metabolism of Carbohydrates': 74, 'Nutrition and Metabolism of Lipids': 86, 'Dietary Reference Standards': 122, 'The Vitamins': 132, 'Minerals and Trace Elements': 188, 'Measuring Food Intake': 238, 'Food Composition': 276, 'Food and Nutrition: Policy and Regulatory Issues': 293, 'Nutrition Research Methodology': 305, 'Food Safety: A Public Health Issue of Growing Importance': 324, 'Food and Nutrition-Related Diseases: The Global Challenge': 350}]}, {'file_name': 'Manual-nutricion-dietetica-CARBAJAL.pdf', 'matching_substrings': {'Algunos conceptos básicos': '6', 'Composición corporal': None, 'Ingestas recomendadas, objetivos nutricionales y guías alimentarias': None, 'Energía': Non

In [157]:
def add_indices_to_keys(data):
    for file_dict in data:
        if isinstance(file_dict['matching_substrings'], list):
            for sub_dict in file_dict['matching_substrings']:
                new_sub_dict = {}
                for idx, (key, value) in enumerate(sub_dict.items(), 1):
                    # If the key doesn't already start with a numeric pattern, prepend it with the index
                    if not re.match(r"^\d+(\.\d+)*", key):
                        new_sub_dict[f"{idx}. {key}"] = value
                    else:
                        new_sub_dict[key] = value
                sub_dict.clear()
                sub_dict.update(new_sub_dict)
        else:
            new_sub_dict = {}
            for idx, (key, value) in enumerate(file_dict['matching_substrings'].items(), 1):
                if not re.match(r"^\d+(\.\d+)*", key):
                    new_sub_dict[f"{idx}. {key}"] = value
                else:
                    new_sub_dict[key] = value
            file_dict['matching_substrings'].clear()
            file_dict['matching_substrings'].update(new_sub_dict)
    return data

all_content = add_indices_to_keys(all_content)

In [158]:
print(all_content)

[{'file_name': 'Introduction to Human Nutrition 2nd Edition ( PDFDrive ).pdf', 'matching_substrings': [{'1. Introduction to Human Nutrition: A Global Perspective on Food and Nutrition': 1, '2. Body Composition': 12, '3. Energy Metabolism': 31, '4. Nutrition and Metabolism of Proteins and Amino Acids': 49, '5. Digestion and Metabolism of Carbohydrates': 74, '6. Nutrition and Metabolism of Lipids': 86, '7. Dietary Reference Standards': 122, '8. The Vitamins': 132, '9. Minerals and Trace Elements': 188, '10. Measuring Food Intake': 238, '11. Food Composition': 276, '12. Food and Nutrition: Policy and Regulatory Issues': 293, '13. Nutrition Research Methodology': 305, '14. Food Safety: A Public Health Issue of Growing Importance': 324, '15. Food and Nutrition-Related Diseases: The Global Challenge': 350}]}, {'file_name': 'Manual-nutricion-dietetica-CARBAJAL.pdf', 'matching_substrings': {'1. Algunos conceptos básicos': '6', '2. Composición corporal': None, '3. Ingestas recomendadas, objetiv

In [159]:
import pprint

pp = pprint.PrettyPrinter(indent=4, sort_dicts= False)
pp.pprint(all_content)

[   {   'file_name': 'Introduction to Human Nutrition 2nd Edition ( PDFDrive '
                     ').pdf',
        'matching_substrings': [   {   '1. Introduction to Human Nutrition: A Global Perspective on Food and Nutrition': 1,
                                       '2. Body Composition': 12,
                                       '3. Energy Metabolism': 31,
                                       '4. Nutrition and Metabolism of Proteins and Amino Acids': 49,
                                       '5. Digestion and Metabolism of Carbohydrates': 74,
                                       '6. Nutrition and Metabolism of Lipids': 86,
                                       '7. Dietary Reference Standards': 122,
                                       '8. The Vitamins': 132,
                                       '9. Minerals and Trace Elements': 188,
                                       '10. Measuring Food Intake': 238,
                                       '11. Food Composition': 27